In [1]:
from experiment import *


import os
# import torch
from cugal.config import SinkhornMethod, HungarianMethod

ModuleNotFoundError: No module named 'FUGAL'

In [39]:
config = Config(
    sinkhorn_method=SinkhornMethod.LOG,
    hungarian_method=HungarianMethod.SCIPY,
    device="cuda",
    dtype=torch.float32, 
    sinkhorn_cache_size=1,
    sinkhorn_regularization=0.2,
    mu=2.0,
    use_sparse_adjacency=True,
    recompute_distance=True,
    frank_wolfe_threshold=None,
)

experiment = Experiment(
    graphs=[
        Graph(GraphKind.NEWMAN_WATTS, {
            'n': 1024*2,
            'k': 16,
            'p': 0.4,
        }),
    ],
    algorithms=[
        Algorithm(config, use_fugal=False),
        # Algorithm(Config(), use_fugal=True),\n",
    ],
    noise_levels=[
        NoiseLevel(0.075, 0.0, False),
    ],
)
results = experiment.run()

folder = "results"
if not os.path.exists(folder): os.makedirs(folder)
# results.dump(folder)\n",

for graph, noise_level, algorithm, result in results.all_results():
    print('graph:', graph)
    print('noise level:', noise_level)
    print('algorithm:', algorithm)
    print()
    print(result)
    print()

λ: 100%|██████████| 15/15 [00:00<00:00, 16.75it/s]


graph: NEWMAN_WATTS {'n': 2048, 'k': 16, 'p': 0.4}
noise level: {'source_noise': 0.075, 'target_noise': 0.0, 'refill_edges': False}
algorithm: Cugal {'safe_mode': False, 'device': 'cuda', 'dtype': 'float32', 'sinkhorn_regularization': 2.0, 'sinkhorn_method': 'STANDARD', 'sinkhorn_iterations': 500, 'sinkhorn_threshold': 0.001, 'sinkhorn_eval_freq': 10, 'mu': 2.0, 'iter_count': 15, 'frank_wolfe_iter_count': 10, 'frank_wolfe_threshold': None, 'use_sparse_adjacency': True, 'sinkhorn_cache_size': 1, 'recompute_distance': True, 'hungarian_method': 'SCIPY'}

Induced Conserved Structure (ICS)  0.009206737062570905
Edge Correctness (EC)              0.009965051478228015
Symmetric Substructure Score (SSS) 0.004808459242040974
Accuracy                           0.001953125
Time (seconds)                     0.9784538745880127

